Antes de cualquier cosa, debes autentificarte a través del cmd, ejecutando:<br>
earthengine authenticate <br>
y antes deberías instalar pip install earthengine-api geemap geopandas pandas

In [1]:
import ee
import geopandas as gpd
import pandas as pd
from datetime import datetime
from shapely.geometry import mapping
import json
from tqdm import tqdm
import csv

Hay que ir a 
https://console.cloud.google.com/apis/dashboard?invt=AbuKWg&project=maximal-osprey-287122
para habilitar la consulta.

In [2]:
# Inicializar Earth Engine
ee.Initialize(project='maximal-osprey-287122')  # Reemplaza con tu ID real

In [3]:
# Cargar shapefile local
shapefile_path = "C:/Users/Michael/Documents/ml_zc/ecu_adm_2024/ecu_adm_adm3_2024.shp"
gdf = gpd.read_file(shapefile_path)

In [4]:
parroquias = gdf[['ADM3_PCODE', 'geometry']]
# Solo selecciono dos filas, para reducir la carga de procesamiento

In [5]:
# parroquias = parroquias[parroquias['ADM3_PCODE'].isin(['EC050651', 'EC010951'])] #'EC050651', 
# parroquias = parroquias[770:]
parroquias = parroquias[parroquias['ADM3_PCODE'] != 'EC160156']

In [6]:
# parroquias = parroquias.iloc[775:]
# Ya que montalvo tiene una geometría muy pequeña, vamos a aumentar por metros
# gdf[gdf['ADM3_PCODE'] == 'EC160156']

In [7]:
# Función para convertir a EE FeatureCollection correctamente
def gdf_to_fc(tabla_geometria):
    features = []
    for _, row in tabla_geometria.iterrows():
        # Convierte la geometría a formato geojson (dict) con mapping()
        geojson_geom = mapping(row['geometry'])
        geom = ee.Geometry(geojson_geom)
        props = row.drop('geometry').to_dict()
        features.append(ee.Feature(geom, props))
    return ee.FeatureCollection(features)

In [8]:
# Función para obtener los metros por polígono de parroquia
def calcular_area_urbana_por_anio(anio, geometria):
    # Cargar la imagen de MODIS para el año especificado
    img = modis.filter(ee.Filter.calendarRange(anio, anio, 'year')).first()
    uso_suelo = img.select('LC_Type1')  # Bandas de clasificación IGBP
    
    # Crear una máscara para la clase urbana (IGBP clase 13)
    urbano = uso_suelo.eq(13)
    
    # Calcular el área urbana (en metros cuadrados por píxel)
    area_pixel = ee.Image.pixelArea().updateMask(urbano)
    
    # Reducir la imagen para calcular el área por parroquia
    stats = area_pixel.reduceRegions(
        collection=geometria,
        reducer=ee.Reducer.sum(),
        scale=500,
    ).first()  # Tomamos solo el primer (y único) valor
    
    # Extraer el valor de la propiedad 'sum' y convertirlo a km²
    area_m2 = stats.get('sum').getInfo()  # Obtiene el área en m²
    #area_km2 = area_m2 / 1e6  # Convertir m² a km²
    
    return area_m2

In [9]:
# Cargar colección de imágenes
modis = ee.ImageCollection('MODIS/006/MCD12Q1')

C:\Users\Michael\AppData\Local\Programs\Python\Python310\lib\site-packages\ee\deprecation.py:207: DeprecationWarning: 

Attention required for MODIS/006/MCD12Q1! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD12Q1

  warnings.warn(warning, category=DeprecationWarning)


In [10]:
# Un ejemplo de la aplicación individual

# Obtener fila de la parroquia
#parroquia = parroquias.iloc[1]

# Convertir la fila (Serie) a un GeoDataFrame
#temp_gdf = gpd.GeoDataFrame([parroquia], geometry='geometry', crs=parroquias.crs)

# Transformamos el objeto a formato compatible con Google Earth Engine
# polygon = gdf_to_fc(temp_gdf)

# Ejemplo de cálculo
# calcular_area_urbana_por_anio(2020, polygon)

In [11]:
# Definir años a procesar
years = list(range(2014, 2025))

# Abrir el archivo en modo escritura (o append si ya existe)
with open('C:/Users/Michael/Documents/ml_zc/datosObtenidos/urbanidad.csv', mode='w', newline='', encoding='utf-8') as file:
    # Crear el escritor CSV
    writer = csv.DictWriter(file, fieldnames=['ADM3_PCODE', 'Año', 'm2'])
    
    # Escribir el encabezado (si es necesario)
    writer.writeheader()
    
    for num_fila in tqdm(range(0, parroquias.shape[0]), desc="Procesando parroquias", ncols=parroquias.shape[0]):
        
        # Obtener fila de la parroquia
        parroquia = parroquias.iloc[num_fila]
    
        # Convertir la fila (Serie) a un GeoDataFrame
        temp_gdf = gpd.GeoDataFrame([parroquia], geometry='geometry', crs=parroquias.crs)
    
        # Transformamos el objeto a formato compatible con Google Earth Engine
        polygon = gdf_to_fc(temp_gdf)

        # Extraemos el código de la parroquia para usarla luego
        name_parr = parroquia['ADM3_PCODE']

        # Mostrar parroquia en proceso
        print(name_parr)

        # Iterar sobre el periodo
        for year in years:
            m2 = calcular_area_urbana_por_anio(2020, polygon)

            # Creamos un diccionario para agregarlos a la lista que se transformará
            row = {'ADM3_PCODE': name_parr, 'Año': year, 'm2': m2}

            writer.writerow(row)

Procesando parroquias:   0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

EC010150


Procesando parroquias:   0%|▉                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

EC010151


Procesando parroquias:   0%|█▉                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

...

Procesando parroquias: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

EC240351


Procesando parroquias: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

EC240352


Procesando parroquias: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████